In [ ]:
#source : https://medium.com/geekculture/introduction-to-the-gan-in-pytorch-bba920347b01

import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as alt
import random

In [ ]:
#binary classifier that determines whether data points are from the original distribution or the fake (generated) distribution
class Discriminator(torch.nn.Module):
  def __init__(self, input_dim=2, hidden_dim=28, n_layers=3):
    super(Discriminator,self).__init__()

    self.input = torch.nn.Sequential( torch.nn.Linear(input_dim, hidden_dim), torch.nn.LeakyReLU() )
    self.layers = []
    for i in range(n_layers):
      self.layers.append( torch.nn.Sequential( torch.nn.Linear(hidden_dim, hidden_dim), torch.nn.LeakyReLU() ) )
    
    self.layers = torch.nn.ModuleList(self.layers)
    self.output = torch.nn.Sequential( torch.nn.Linear(hidden_dim, 1), torch.nn.Sigmoid() )

  def forward(self, x):
    o = self.input(x)
    for layer in self.layers:
      o = layer(o)

    o = self.output(o)
    return o
  
#Generator, takes in random noise and outputs a fake point
class Generator(torch.nn.Module):
  def __init__(self, z_dim=1, hidden_dim=28, n_layers=3, out_dim=2):
    super(Generator, self).__init__()

    self.input = torch.nn.Sequential( torch.nn.Linear(z_dim, hidden_dim), torch.nn.LeakyReLU() )
    self.layers = []
    for i in range(n_layers):
      self.layers.append( torch.nn.Sequential( torch.nn.Linear(hidden_dim, hidden_dim ), torch.nn.LeakyReLU() ) )
    
    self.layers = torch.nn.ModuleList(self.layers)
    self.output = torch.nn.Linear(hidden_dim, out_dim)

  def forward(self, x):
    o = self.input(x)
    for layer in self.layers:
      o = layer(o)

    o = self.output(o)
    return o

In [ ]:
#sample n points from [-range/2,range/2] from distribution
#dist is a single parameter function
def sample_dist(n, r, dist, mode='1d'):
  data = []

  if mode == '1d':
    x = r*(np.random.random_sample((n,))-0.5)
    
    for i in range(n):
          data.append([x[i], dist(x[i])])
  elif mode == '2d':
    for i in range(n):
      #2d dist auto samples from x,y
      out = dist()
      data.append([out[0], out[1]])

  return np.array(data)

#sample noise uniformly from -1 to 1, n vectors with dimensionality m
#m is the dimension of the latent space
def sample_noise(n, m):
  return np.random.uniform(-1., 1., size=[n, m])

In [ ]:
def parabolic_dist(x):
  return x*x

def inverse_parabolic_dist(x):
  return -(x*x)

def gauss_2d(mu=0, sigma=1):
    x = random.gauss(mu, sigma)
    y = random.gauss(mu, sigma)
    return (x, y)

def circle():
    theta = random.random() * 2 * np.pi
    x = np.cos(theta)
    y = np.sin(theta)
    return (x, y)

def square():
    r = random.random() * 4
    pos = random.random() - 0.5
    if r > 3:
      return (0.5, pos)
    if r > 2:
      return (-0.5, pos)
    if r > 1:
      return (pos, 0.5)
    else :
      return (pos, -0.5)

def heart():
    t = random.random() * 2 * np.pi;
    x = 16 * np.sin(t) * np.sin(t) * np.sin(t)
    y = 13 * np.cos(t) - 5 * np.cos(2 * t) - 2 * np.cos(3 * t) - np.cos(4 * t)
    return (x, y)

def infinity():
    t = (random.random() * 2 * np.pi) - 0.5 * np.pi;
    x = np.cos(t)
    y = np.cos(t) * np.sin(t)
    return (x, y)


In [ ]:
def train(gen, disc, optimG, optimD, distribution, criterion, fixed_noise, r=50, z_dim=10, num_epochs=30000, batch_size=32, mode='1d', device='cuda'):
  real_label = 1
  fake_label = 0

  gen_list = []
  G_losses = []
  D_losses = []
  iters = 0

  print("Training...")
  print(device)
  for epoch in range(num_epochs):
        #max log(D(x)) + log(1 - D(G(z)))
        #train on real points
        disc.zero_grad()
        
        real_points = torch.tensor( sample_dist(n=batch_size, r=r, dist=distribution, mode=mode) ).float()
        label = torch.full( (batch_size,), real_label, dtype=torch.float, device=device ).view(-1)

        output = disc(real_points).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()

        #train on fake points
        noise = torch.randn(batch_size, z_dim, device=device)
        fake_points = gen(noise)
        label.fill_(fake_label)
        output = disc(fake_points.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        
        errD = errD_real + errD_fake

        optimizerD.step()

        #max log(D(G(z)))
        gen.zero_grad()
        label.fill_(real_label)

        output = disc(fake_points).view(-1)

        errG = criterion(output, label)

        errG.backward()

        optimizerG.step()

        # Output training stats
        if iters % 50 == 0:
            print('[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f'
                  % (epoch, num_epochs, errD.item(), errG.item()))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1)):
            with torch.no_grad():
                fake = gen(fixed_noise).detach().cpu()
            gen_list.append(fake)

        iters += 1

  gen_list = np.array([np.array(x) for x in gen_list])
  return gen, disc, G_losses, D_losses, gen_list

In [ ]:
def plot_losses(G_losses, D_losses):
  #plot losses
  plt.figure()
  plt.title('Generator and Discriminator Losses')
  plt.plot(range(len(G_losses)), G_losses, color='green')
  plt.plot(range(len(D_losses)), D_losses, color='red')

def make_anim(gen_list, distribution, r, mode='1d'):

  if mode == '1d':
    plt.rc('animation', html='jshtml')

    def update(i):
      ax.set_xlabel('Step: {}'.format(i))
      xf = [t[0] for t in gen_list[i]]
      yf = [t[1] for t in gen_list[i]]
      ax.clear()
      ax.set_xlim((-r*2,r*2))
      ax.scatter(range(-r,r), reals, color='blue', zorder=30)
      ax.scatter(xf, yf, color='red')
      ax.set_aspect('equal')

      return ax
      
    fig, ax = plt.subplots()
    fig.set_tight_layout(False)

    reals = [distribution(x) for x in range(-r,r)]
    anim = alt.FuncAnimation(fig, update, frames=np.arange(0, gen_list.shape[0]), interval=200)

    #plot final generated pairs vs real distribution
    xf = [t[0] for t in gen_list[gen_list.shape[0]-1]]
    yf = [t[1] for t in gen_list[gen_list.shape[0]-1]]

    plt.figure()
    plt.title('Final Generator Fitting')
    plt.scatter(range(-r,r), reals, color='blue')
    plt.scatter(xf, yf, color='red', zorder=30)
    plt.axis('scaled')



  elif mode == '2d':
    plt.rc('animation', html='jshtml')

    def update(i):
      ax.set_xlabel('Step: {}'.format(i))
      xf = [t[0] for t in gen_list[i]]
      yf = [t[1] for t in gen_list[i]]
      ax.clear()
      ax.set_xlim((min(xr),max(xr)))
      ax.scatter(xr, yr, color='blue')
      ax.scatter(xf, yf, color='red', zorder=30)
      ax.set_aspect('equal')


      return ax
      

    fig, ax = plt.subplots()
    fig.set_tight_layout(False)

    reals = [distribution() for x in range(r)]
    anim = alt.FuncAnimation(fig, update, frames=np.arange(0, gen_list.shape[0]), interval=200)

    #plot final generated pairs vs real distribution
    xf = [t[0] for t in gen_list[gen_list.shape[0]-1]]
    yf = [t[1] for t in gen_list[gen_list.shape[0]-1]]

    xr = [t[0] for t in reals]
    yr = [t[1] for t in reals]

    plt.figure()
    plt.title('Final Generator Fitting')
    plt.scatter(xr, yr, color='blue')
    plt.scatter(xf, yf, color='red', zorder=30)
    plt.axis('scaled')


  return anim

In [ ]:
#train to fit simple function x^2
z_dim = 10
num_epochs = 30000
batch_size = 32

device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

criterion = torch.nn.BCELoss()
generator = Generator(z_dim=z_dim, hidden_dim=28, n_layers=3, out_dim=2).to(device)
discriminator = Discriminator(input_dim=2, hidden_dim=28, n_layers=3).to(device)

optimizerD = torch.optim.Adam(discriminator.parameters())
optimizerG = torch.optim.Adam(generator.parameters())
fixed_noise = torch.randn(128, z_dim, device=device)

distribution = infinity
r = 512 
#r = 25
mode = '2d'
#mode = '1d'

generator, discriminator, gloss, dloss, gen_list = train(generator, discriminator, optimizerG, optimizerD, distribution, criterion, fixed_noise, r=50, z_dim=z_dim, num_epochs=num_epochs, batch_size=batch_size, device=device, mode=mode)

In [ ]:
plot_losses(gloss, dloss)

In [ ]:
make_anim(gen_list, distribution, r, mode=mode)
